In [4]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib
# import sklearn
import torch

from torch import nn, optim

from IPython.display import display
# plt.style.use("ggplot")

print("package版本信息：")
print("numpy:      ", np.__version__)
# print("pandas:     ", pd.__version__)
# print("matplotlib: ", matplotlib.__version__)
# print("sklearn:    ", sklearn.__version__)
print("PyTorch:     ", torch.__version__)

package版本信息：
numpy:       1.24.3
PyTorch:      1.12.0


# Pytorch-Transformer

参考文档:
+ [Pytorch tutorial -> Transformer tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)
+ [Pytorch -> Transformer Layers](https://pytorch.org/docs/stable/nn.html#transformer-layers)

这里介绍了Pytorch里 transformer 层的实现模块.

主要有如下几个类：
+ `nn.Transformer`：一步到位构建整个Transformer模型，底层使用的是下面的API
+ `nn.TransformerEncoder`：Encoder层，包含了多个 `TransoformerEncoderLayer`
+ `nn.TransformerDecoder`：Decoder层，包含了多个 `TransformerDecoderLayer`
+ `nn.TransformerEncoderLayer`：单层Encoder
+ `nn.TransformerDecoderLayer`：单层Decoder

但是需要注意的是，上述各种层的**底部都是调用的`activation.py`中的`MultiheadAttention`类**。

## Multi-Head Attention Layer

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html#torch.nn.MultiheadAttention).

它是`TransformerEncoderLayer`和`TransformerDecoderLayer`层中 **self-attention 层**的实现。  
需要注意的是，pytorch源码中 `MultiheadAttention` 被放在了 Non-linear Activations 这一类中（并且是放在`activation.py`文件中的）。


### 实例化参数
`MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None)`  
+ `embed_dim`，整个MHA的输入/输出特征维度。  
  **原始论文里，MHA的输入/输出特征维度可以是不一样的，但是在Pytorch的实现里，强制要求它们是一样的**（这个限制未来版本可能会取消）。
+ `num_heads`，head数量，注意，分配到每个head的维度 = embed_dim/num_heads.
+ `kdim`，自定义 key 中的特征维度，它是下面`forward()`方法里`key`对应的特征维度，默认下=`embed_dim`
+ `vidm`，自定义 value 中的特征维度，`forward()`方法里`value`对应的特征维度，默认=`embed_dim`
+ `bias`，
+ `add_bias_kv`
+ `add_zero_attn`
+ `batch_first`，早期版本的pytorch中，会要求下面`forward()`方法里输入数据的各个维度顺序必须为：`(序列长度，batch_size, embedding_dimension)`，后来提供了这个`batch_first`参数，用于适应一下输入数据的维度变化.


### 前向传播

`MultiheadAttention.forward(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None)`
+ `query`, `key`, `value`，MHA的三个输入矩阵——这三个输入向量会被用于生成对应的Q,K,V
  + `query.shape` = $(L, N, E_q)$，`key.shape` = $(S, N, E_k)$，`value.shape` = $(S, N, E_v)$.   
    + $N$ 是 batch size
    + $E$ 是 embedding 维度，$E_q$=`embed_dim`, $E_k$=`kdim`, $E_v$=`vdim`
    + $L$ 是**target**序列的长度——因为它决定了MHA输出的序列长度
    + $S$ 是**source**序列的长度
  + Transformer中MHA会**在3个地方使用，这3个地方会影响 query, key, value 的内容**
    + Encoder中 query, key, value 都是来自于样本的**输入序列**，此时 $L=S$，`kdim == vdim`
    + Decoder的 **Masked MHA** 中 query, key, value 都来自于样本的**输出序列**，此时 $L=S$, `kdim == vdim`
    + Decoder的 **decoder-encoder attention**中，query 来自于样本的**输出序列**，key, value 都是**Encoder最后一层的输出**，此时可能 $L\neq S$, `kdim != vdim`
  + **有关序列维度的说明**
    + 输入的 `query`, `key`, `value` 不是self-attention的输入，它们需要经过各自的投影矩阵 $W_Q \in (E_q, E_q)$, $W_K \in (E_d, E_k)$, $W_V \in (E_d, E_v)$ 的转换，上面这3个参数矩阵的维度，就是MHA里初始化的维度
    + 线性投影过程如下（这里把batch的维度提前了）：   
    $q = query * W_Q^T \in (N, L, E_d)$,   
    $k = key * W_K^T \in (N, S, E_d)$,   
    $v = value * W_V^T \in (N, S, E_d)$  
    + 可以看出，参数的 `kdim` 和 `vdim` 只是用于设置投影矩阵 $W_K, W_V$ 的维度，这两个矩阵的另一个维度被设置成了 $E_q$=`embed_dim`，经过上述三个投影矩阵的变换之后，`query`, `key`, `value` 的维度都变成了 $E_q$，也就是 `embed_dim`
    + 后续进行的MHA操作为：
    $${\rm softmax}(\frac{q*k^T}{\sqrt{E_d}}) * v$$
    $$((N, L, E_d) * (N, E_d, S)) * (N, S, E_d) \rightarrow (N, L, E_d)$$
    调用的是`torch.bmm()`算子，会在每个批次上做上述操作。
    + 可以看出，最终决定输出序列长度的是 `query` 的序列长度 $L$，而 `key` 和 `value` 的序列长度必须一致，它们会在中间计算过程中相互抵消掉。


+ `key_padding_mask`：填充位置的掩码，shape=$(N, S)$   
  + 对于一个batch的训练数据来说，batch size 为 $N$ 表示有 $N$ 个序列，但是**每个序列的长度（也就是单词个数）是不一样的**，`key`,`value`中的 $S$ 都是指最长序列的长度，其他的序列就需要对缺少的单词位置做填充(padding)，padding位置的词通常使用全为0的embeding 来表示。   
  + 为了不让MHA对这些位置的词产生注意力，需要记录这些padding位置，每个batch都使用这样一个 $(N, S)$ 的矩阵来标识padding的位置，该矩阵的每一行对应batch中的一个序列，值为`[False, False, True, True]` 或者 `[0, 0, 1, 1]` 的形式，其中结尾`True`或者1标识的就是padding的位置。   
  + 需要注意的是，这里padding矩阵实际生效位置是在`query`和`key`做完矩阵乘法得到shape=$(N,L,S)$的中间矩阵 P 之后才使用的，由于只有`key`使用了padding掩码，只需要在 $N$ 个序列中对 $S$ 这一个维度标识padding位置即可，所以使用的padding矩阵shape=$(N,S)$. pytorch中的实现就是对中间矩阵 P 的 $S$ 维度，按照 `[False,False,True,True]` 中的标记，对应`True`的位置赋值为`-inf`，从而在后续的softmax中，该位置得到的概率为0.


+ `attn_mask`，**只在Decoder中使用**，主要是解码时不让当前位置的词和后面要预测的词序列进行Attention.   
  + shape=$(L,S)$ 或者 $(N * num\_heads,L,S)$，如果是 2D 的矩阵，那么batch中的每个序列都会使用这个，如果是 3D 的矩阵，则会指定每个batch 的掩码
  + 它的生效位置也是在`query`和`key`相乘得到shape=$(N,L,S)$的中间矩阵 P 之后使用，此时batch中**每个**序列的输入`query`部分（shape=$(L,E)$）和`key`部分（shape=$(S,E)$）相乘得到了一个 $(L,S)$ 的矩阵 P，矩阵中列表示`query`里的每个词，行表示`key`中的每个词，如果不想让`query`中第 $i$ 个词看到`key`中第 $j$ 个词之后的内容，只需要让 `P[i, j:]` 这部分为 `-inf` 即可，因此这个掩码的shape=$(N,L,S)$.
  + 如果是在Decoder中的**Masked MHA**这里使用，此时输入的`query`,`key`,`value`都是来源于同一个地方，`query`和`key`维度相等，此时 P 是一个方阵，并且右上三角均为`-inf` —— 这是好理解的部分，也是正常它要解决的问题部分。
  + 如果是在Decoder中的**encoder-decoder attention**层使用，此时输入的`query`是来自于之前的Masked MHA层，属于输出序列的内容，但是`key`,`value`是来自于Encoder，属于输入序列的内容，此时才会出现`query`和`key`长度不等的情况——这种情况下虽然也可以使用掩码，但是它的含义就不那么好解释了。


+ `need_weights`，bool，表示是否需要返回`attn_output_weigths`，默认为`True`


+ `forward()`方法返回值
  + `attn_output`，shape= $(L, N, E_q)$，$L$ 由 输入的`query`序列长度决定，$E$=`embed_dim`.
  + `attn_output_weights`，shape=$(N, L, S)$，这个权重应该是每个输入序列中，每个word的attention权重.
  

### 一些说明

Pytorch中的Multi-Head Attention的实现和论文《All you need is transformer》中有一些不一样的地方，总结如下：

+ pytorch在实现MHA时，强制要求了输入的特征维度和输出的特征维度必须一致，也就是实例化时的`embed_dim`参数，但是在原论文中没有这个限制。   
原论文中，输入的特征维度是`query`里的embed_dim，输出的特征维度由`value`的embed_dim决定。

+ pytorch中MHA实例化参数 `kdim`,`vdim` 和含义和原论文里的 $d_k$, $d_v$ 不一样，如下所示.   
原论文中的 $d_k$, $d_v$ 指的是 query, key 经过参数矩阵 $W$ 变换之后的 embed_dim，而pytoch中的 `kdim`,`vdim` 指定的是输入的 `key`,`value` 中，没有经过 $W$ 变换之前的特征维度，$W$ 这个参数矩阵设置的时候，其中一个维度就是 `kdim` （或者 `vdim`），另一个维度就是 `embed_dim`，因此 `key`, `value` 经过 $W$ 变换之后，维度就变成了 `embed_dim` ——这是 pytorch强制规定的。

<img src="images/multi-head-attention-formula.png" width="60%" align="left">

### 使用示例

In [2]:
embed_dim, num_heads = 512, 8
batch_size, target_len, source_len = 5, 10, 20

query = torch.rand(target_len, batch_size, embed_dim)
key = torch.rand(source_len, batch_size, embed_dim)
value = torch.rand(source_len, batch_size, embed_dim)

In [3]:
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
attn_output, attn_output_weights = multihead_attn(query, key, value)
print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


In [4]:
kdim, vdim = 256, 256
key = torch.rand(source_len, batch_size, kdim)
value = torch.rand(source_len, batch_size, vdim)

multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, kdim=kdim, vdim=vdim)
attn_output, attn_output_weights = multihead_attn(query, key, value)

print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


## Encoder Layer

### 单层Encoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html#torch.nn.TransformerEncoderLayer)，它内部包含了一个 self-attention 层 + 一个 feedforward 层.

`nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`  

+ 实例化参数
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1  


+ 前向传播`forward(src, src_mask=None, src_key_padding_mask=None)`
  + `src`，输入的sequence, `shape` = $(S, N, E)$
  + `src_mask`，输入sequence的mask
  + `src_key_padding_mask`，每个batch中src的padding矩阵  


+ `forward()`方法返回值  
  它返回的是和 `src` shape 相同的 tensor.


Encoder的内部只有**一层**`MultiheadAttention`：  
`Encoder.forward()`实际执行时，调用方式为`MultiheadAttention.forward(src, src, src)`——`src`会同时作为 query, key, value 传入 self-attention。



### 多层Encoder

`TransformerEncoder(encoder_layer, num_layers, norm=None)`
+ 参数：
  + `encoder_layer`，是一个`TransformerEncoderLayer`类的实例对象
  + `num_layers`，指定堆叠的 Encoder 层数
+ `forward()`方法和`TransformerEncoderLayer`的一致

In [5]:
d_model, num_heads, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
src = torch.rand(source_len, batch_size, d_model)

encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=dim_ffn)
out = encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


In [8]:
multi_encoder_layer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=3)
out = multi_encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


## Decoder Layer

### 单层Decoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerDecoderLayer.html#torch.nn.TransformerDecoderLayer)

`TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`   

+ 实例化参数：和 Encoder 一致
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1


+ 前向传播`forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`  
  这个方法和 Encoder 不一样，它多了一个 memory. 
  + `tgt`，输入的序列，`shape`= $(S, N, E)$，
  + `memory`，来自 encoder 层（通常是最后一层）的输出, `shape`=$(S, N, E)$.
  + `tgt_mask`，
  + `memory_mask`
  + `tgt_key_padding_mask`，
  + `memory_key_padding_mask`

> `Decoder`内部会调用两层`MultiheadAttention`
> 1. 第一层调用时为`MultiheadAttention.forward(tgt, tgt, tgt)`
> 2. 第二层调用时为`MultiheadAttention.forward(tgt, memory, memory)`


+ `forward()`方法返回值  
它返回的是和 `tgt` shape 相同的 tensor，


### 多层Decoder

`TransformerDecoder(decoder_layer, num_layers, norm=None)`   
参数同 `TransformerEncoder` 类.

In [7]:
d_model, nhead, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
tgt = torch.rand(source_len, batch_size, d_model)
memory = torch.rand(source_len, batch_size, d_model)

decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_ffn)
decoder_out = decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


In [9]:
multi_decoder_layer = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=3)
decoder_out = multi_decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


## Transformer类

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#torch.nn.Transformer)

`nn.Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1, activation=<function relu>, custom_encoder=None, custom_decoder=None, layer_norm_eps=1e-05, batch_first=False, norm_first=False, device=None, dtype=None)`

+ 实例化参数
  + `d_model`，输入/输出的embedding dim，默认512
  + `nhead`
  + `num_encoder_layers`
  + `num_decoder_layers`
  + `dim_feedforward`
  + `dropout`
  + `activation`
  
+ 正向传播`forward(src, tgt, src_mask=None, tgt_mask=None, memory_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`
  + `src`，shape=$(S,N,E)$
  + `tgt`，shape=$(T,N,E)$
  + `src_mask`，shape=$(S,S)$
  + `tgt_mask`，shape=$(T,T)$
  + `memory_mask`，shape=$(T,S)$
  + `src_key_padding_mask`，shape=$(N,S)$
  + `tgt_key_padding_mask`，shape=$(N,T)$
  + `memory_key_padding_mask`，shape=$(N,S)$